In [10]:
import sys
import os
import copy

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from get_data import get_data_from_url
url = 'https://adventofcode.com/2024/day/6/input'

raw_data = get_data_from_url(url)
raw_data_list_of_strings = raw_data.splitlines()
raw_data_list_of_lists = [list(a_string) for a_string in raw_data_list_of_strings]

In [11]:
guard_possible_facings = ['^', '>','v','<']

def identify_position(position_and_obstacles_map):
    for row_num, row in enumerate(position_and_obstacles_map):
        guard_position = None
        for guard_facing in guard_possible_facings:
            if (guard_facing in row):
                guard_position = [row_num, row.index(guard_facing)]
                return guard_position, guard_facing
            
def identify_next_movement_position(position_and_obstacles_map):
    guard_position, guard_facing = identify_position(position_and_obstacles_map)
    guard_new_position = ['','']
    if guard_facing == '^':
        guard_new_position[0] = guard_position[0]-1
        guard_new_position[1] = guard_position[1]
    elif guard_facing == 'v':
        guard_new_position[0] = guard_position[0]+1 
        guard_new_position[1] = guard_position[1]
    elif guard_facing == '>':
        guard_new_position[0] = guard_position[0] 
        guard_new_position[1] = guard_position[1]+1
    elif guard_facing == '<':
        guard_new_position[0] = guard_position[0] 
        guard_new_position[1] = guard_position[1]-1
    return guard_new_position, guard_facing


def there_is_an_obstacle_ahead(position_and_obstacles_map, guard_new_position):
    if position_and_obstacles_map[guard_new_position[0]][guard_new_position[1]]=='#':
        return True
    else:
        return False
    
def turn_right_and_update_map_guard_facing_direction(position_and_obstacles_map,guard_position):
    guard_turning_right = ['>','v','<','^' ] 
    guard_facing = position_and_obstacles_map[guard_position[0]][guard_position[1]]
    guard_turned_right = guard_turning_right[guard_possible_facings.index(guard_facing)]
    updated_position_and_obstacles_map = copy.deepcopy(position_and_obstacles_map)
    updated_position_and_obstacles_map[guard_position[0]][guard_position[1]]=guard_turned_right
    return updated_position_and_obstacles_map

def guard_will_be_in_area(position_and_obstacles_map, guard_new_position):
    max_num_of_rows = len(position_and_obstacles_map)
    max_num_of_columns = len(position_and_obstacles_map[0])
    if (guard_new_position[0] < max_num_of_rows) and (guard_new_position[0] > -1):
        if (guard_new_position[1] < max_num_of_columns) and (guard_new_position[1] > -1):
            return True
        else:
            return False
    else: 
        return False
    
def mark_position_with_X_update_map(position_and_obstacles_map, current_position):
    current_position_and_obstacles_with_Xs = copy.deepcopy(position_and_obstacles_map)
    
    current_position_and_obstacles_with_Xs[current_position[0],current_position[1]]='X'
    return current_position_and_obstacles_with_Xs

def mark_new_position_with_guard_facing_direction(position_and_obstacles_map):
    updated_position_and_obstacles_map = copy.deepcopy(position_and_obstacles_map)
    guard_new_position, guard_facing = identify_next_movement_position(position_and_obstacles_map)
    updated_position_and_obstacles_map[guard_new_position[0]][guard_new_position[1]] = guard_facing
    return updated_position_and_obstacles_map

def mark_new_position_dir_and_old_position_x_in_map(position_and_obstacles_map, current_position):
    next_position, guard_facing = identify_next_movement_position(position_and_obstacles_map)

    updated_position_and_obstacles_map = copy.deepcopy(position_and_obstacles_map)
    updated_position_and_obstacles_map[current_position[0]][current_position[1]] = 'x'
    updated_position_and_obstacles_map[next_position[0]][next_position[1]] = guard_facing
    return updated_position_and_obstacles_map
    
def move_guard_to_next_step_update_map_with_x_and_new_position(position_and_obstacles_map):
    cloned_position_and_obstacles_map = copy.deepcopy(position_and_obstacles_map)
    current_position, _ = identify_position(cloned_position_and_obstacles_map)
    probable_next_position, _ = identify_next_movement_position(cloned_position_and_obstacles_map)
    if not guard_will_be_in_area(cloned_position_and_obstacles_map, probable_next_position):
        return cloned_position_and_obstacles_map

    elif there_is_an_obstacle_ahead(cloned_position_and_obstacles_map, probable_next_position):
        map_with_guard_facing_right_way = turn_right_and_update_map_guard_facing_direction(cloned_position_and_obstacles_map,current_position)
    else: 
        map_with_guard_facing_right_way = cloned_position_and_obstacles_map
    updated_map = mark_new_position_dir_and_old_position_x_in_map(map_with_guard_facing_right_way, current_position)
    return updated_map

calculate_first_answer = False
if calculate_first_answer == True:
    updated_map = move_guard_to_next_step_update_map_with_x_and_new_position(raw_data_list_of_lists)
    rep=0
    found_final_map = False
    while (rep < 100000) and (not found_final_map):
        rep+=1
        if rep%150==0:
            print (rep)
        new_updated_map = move_guard_to_next_step_update_map_with_x_and_new_position(updated_map)

        if updated_map == new_updated_map:
            found_final_map = True
        else:
            updated_map = new_updated_map
            
    def get_answer_a():
        first_list = updated_map
        x=0
        for i, a_list in enumerate(first_list):
            for element in a_list:
                if element == 'x':  
                    x +=1
        return x

    get_answer_a()

    tried1 = 4981 #too low
    tried2 = tried1 + 1  #(last position does not update)  <-correct

In [20]:
def copy_map_remove_guard(a_map_with_obstacles):
    cloned_map = copy.deepcopy(a_map_with_obstacles)
    for row in cloned_map:
        for i,element in enumerate(row):
            if (element == '^') or (element == '>') or (element == 'v') or (element == '<') :
                row[i] = '.'
    return cloned_map

def replace_obstacles_with_empty_lists(position_map_with_obstacles):
    new_position_map = copy.deepcopy(position_map_with_obstacles)
    for row in new_position_map:
        for i,element in enumerate(row):
            if element == '#':
                row[i] = []
    return new_position_map

# def position_in_map(a_map, position):
#     if 


def get_coordinates_for_spot_ahead(guard_position, guard_facing):
    guard_new_position = ['','']
    if guard_facing == '^':
        guard_new_position[0] = guard_position[0]-1
        guard_new_position[1] = guard_position[1]
    elif guard_facing == 'v':
        guard_new_position[0] = guard_position[0]+1 
        guard_new_position[1] = guard_position[1]
    elif guard_facing == '>':
        guard_new_position[0] = guard_position[0] 
        guard_new_position[1] = guard_position[1]+1
    elif guard_facing == '<':
        guard_new_position[0] = guard_position[0] 
        guard_new_position[1] = guard_position[1]-1
        
    return guard_new_position

def guard_will_get_out_of_field_next_step(obstacle_map, current_position_coordinates, direction):
    next_step_coordinates = get_coordinates_for_spot_ahead(current_position_coordinates, direction)
    row = next_step_coordinates[0]
    col = next_step_coordinates[1]
    if (row<0) or (row>(len(obstacle_map)-1)):
        return True
    elif (col<0) or (col>(len(obstacle_map[0])-1)):
        return True
    else:
        return False
    
def no_obstacle_in_position(position_coordinates, obstacle_map):
    position_row = position_coordinates[0]
    position_col = position_coordinates[1]
    if '#' in obstacle_map[position_row][position_col]:
        return False
    else:
        return True
    
def add_obstacle_to_map(obstacles_map, spot_coordinates):
    if (obstacles_map[spot_coordinates[0]][spot_coordinates[1]] != '.'):
        raise ValueError('function add_obstacle_to_map tried to add an obstacle on top of the guard.')
    obstacles_map[spot_coordinates[0]][spot_coordinates[1]] = '#'
    return None

def add_list_to_map(obstacles_map, spot_coordinates):
    if (obstacles_map[spot_coordinates[0]][spot_coordinates[1]] != '.'):
        raise ValueError('function add_list_to_map tried to add a list on top of the guard.')
    obstacles_map[spot_coordinates[0]][spot_coordinates[1]] = []
    return None

turn_right = {
    '^':'>','>':'v','v':'<','<':'^'
}

obstacle_hitting_side = {
    '^':['down'],'>':['left'],'v':['up'],'<':['right']    
}



def update_obstacle_history_map(obstacle_history_map, obstacle_coordinates, \
                            guard_coming_from):
    obstacle_row = obstacle_coordinates[0]
    obstacle_col = obstacle_coordinates[1]
    obstacle_history_map[obstacle_row][obstacle_col]+=obstacle_hitting_side[guard_coming_from]
    return None
    


def get_moving_row_or_col_from_map_as_a_list(a_map_with_obstacles,
                                                position_coordinates, \
                                                direction):
    if direction in ['<','>']:
        return  a_map_with_obstacles[position_coordinates[0]]
    
    elif direction in ['^','v']:
        return  [a_map_with_obstacles[i][position_coordinates[1]] for i in range(0,len(a_map_with_obstacles))] 


def there_is_obstacle_ahead(a_map_with_obstacles, \
                            position_coordinates, \
                            direction):
    position_row=position_coordinates[0]
    position_col=position_coordinates[1]
    moving_row_or_col = get_moving_row_or_col_from_map_as_a_list(a_map_with_obstacles,
                                                position_coordinates, \
                                                direction)
    indexes_of_objects_in_moving_row_or_col = [i for i in range (len(moving_row_or_col)) if moving_row_or_col[i]=='#']
    if direction == '^':
        obstacle_indexes = [i for i in indexes_of_objects_in_moving_row_or_col if i<position_row]
    elif direction == '>':
        obstacle_indexes = [i for i in indexes_of_objects_in_moving_row_or_col if i>position_col]
    elif direction == 'v':
        obstacle_indexes = [i for i in indexes_of_objects_in_moving_row_or_col if i>position_row]
    elif direction == '<':
        obstacle_indexes = [i for i in indexes_of_objects_in_moving_row_or_col if i<position_col]     
    if len(obstacle_indexes)>0:
        return True
    else:
        return False
    
        

def get_next_obstacle_ahead_coordinates(a_map_with_obstacles, \
                                        position_coordinates, \
                                        direction):
    position_row=position_coordinates[0]
    position_col=position_coordinates[1]
    moving_row_or_col = get_moving_row_or_col_from_map_as_a_list(a_map_with_obstacles,
                                                position_coordinates, \
                                                direction)
    indexes_of_objects_in_moving_row_or_col = [i for i in range (len(moving_row_or_col)) if moving_row_or_col[i]=='#']       
    if direction == '^':
        obstacle_col = position_col
        obstacle_row = [i for i in indexes_of_objects_in_moving_row_or_col if i<position_row][-1]
    elif direction == '>':
        obstacle_row = position_row
        obstacle_col = [i for i in indexes_of_objects_in_moving_row_or_col if i>position_col][0]
    elif direction == 'v':
        obstacle_col = position_col
        obstacle_row = [i for i in indexes_of_objects_in_moving_row_or_col if i>position_row][0]
    elif direction == '<':
        obstacle_row = position_row
        obstacle_col = [i for i in indexes_of_objects_in_moving_row_or_col if i<position_col][-1]
    return [obstacle_row,obstacle_col]





def hit_same_obstacle_from_same_side_before(imaginary_obstacles_with_history_map, \
                                            next_obstacle_position_coordinates, hitting_obstacle_from
                                            ):
    obstacle_row = next_obstacle_position_coordinates[0]
    obstacle_col = next_obstacle_position_coordinates[1]
    obstacle_hitting_history_list = imaginary_obstacles_with_history_map[obstacle_row][obstacle_col]
    if hitting_obstacle_from in obstacle_hitting_history_list:
        return True
    else:
        return False
    
def get_new_guard_coordinates_before_next_obstacle_turned_right(next_obstacle_coordinates, 
                                                direction):
    guard_new_row = next_obstacle_coordinates[0]
    guard_new_col = next_obstacle_coordinates[1]
    if direction == '^':
        guard_new_row = guard_new_row+1
    elif direction == '>': 
        guard_new_col = guard_new_col-1
    elif direction == 'v': 
        guard_new_row = guard_new_row-1
    elif direction == '<': 
        guard_new_col = guard_new_col+1
    return [guard_new_row, guard_new_col]
 
# def take_next_long_step(next_obstacle_coordinates, \
#                    imaginary_obstacles_with_history_map, direction):
#     guard_coming_from = obstacle_hitting_side[direction]
#     update_obstacle_history_map(imaginary_obstacles_with_history_map, next_obstacle_coordinates, \
#                                 guard_coming_from)
#     updated_guard_position = get_new_guard_coordinates_before_next_obstacle_turned_right(next_obstacle_coordinates, 
#                                                 direction)
#     updated_direction = turn_right[direction]
    
#     return updated_guard_position, updated_direction

In [39]:
# Initialize variables:


initial_position_map = raw_data_list_of_lists

def check_map(initial_position_map=initial_position_map, debug=False):
    
    real_obstacles_map = copy_map_remove_guard(initial_position_map)
    real_obstacles_history_map = replace_obstacles_with_empty_lists(real_obstacles_map)

    im_obstacles_map = copy_map_remove_guard(initial_position_map)
    im_obstacles_history_map = replace_obstacles_with_empty_lists(real_obstacles_map)

    all_im_obstacles_position_history = []
    
    position, direction = identify_position(initial_position_map)
    starting_position, _ = identify_position(initial_position_map)
    last_real_position, last_real_direction = identify_position(initial_position_map)

    num_of_positions_that_trap_guard = 0

    imaginary_mode = False
    if debug:
        print(f'----0-----{imaginary_mode=}')
    next_obstacle_coordinates = []
    next_obstacle_history = []
    next_obstacle_hitting_site = None

    im_obstacle_coordinates = []

    max_num_of_real_steps = 2*4982
    if debug:
        max_num_of_real_steps = 15
    num_of_real_steps_checked = 0

    turns_lim = 3

    while (not guard_will_get_out_of_field_next_step(real_obstacles_map, position, direction) and \
           num_of_real_steps_checked < max_num_of_real_steps):
        if num_of_real_steps_checked%700 ==0: # or num_of_real_steps_checked>4982:
            print (f'{num_of_real_steps_checked=}')  
        if debug:
            print (f'{direction=},{position=}')

        if not imaginary_mode:
            next_spot_coordinates = get_coordinates_for_spot_ahead(position, direction)
            if debug:
                print (f'real guard at {position=} looking towards {direction}, towards {next_spot_coordinates}')
            times_turned = 0

            while (not no_obstacle_in_position(next_spot_coordinates, real_obstacles_map)):
                if debug:
                    print ('obstacle ahead')


                times_turned+=1
                if times_turned ==3:
                    raise ValueError('Guard turned too many times.')
                update_obstacle_history_map(obstacle_history_map=real_obstacles_history_map, \
                                            obstacle_coordinates=next_spot_coordinates, \
                                            guard_coming_from=direction)
                update_obstacle_history_map(obstacle_history_map=im_obstacles_history_map, \
                                            obstacle_coordinates=next_spot_coordinates, \
                                            guard_coming_from=direction)
                direction = turn_right[direction]
                if debug:
                    print (f'turned {direction}')
                last_real_position, last_real_direction = position, direction
                # next_obstacle_hitting_site = obstacle_hitting_side[direction]
                next_spot_coordinates = get_coordinates_for_spot_ahead(position, direction)


            if debug:
                print ('>>>>>>>>>no obstacle ahead, adding one<<<<<<<')
            im_obstacle_coordinates = next_spot_coordinates
            if im_obstacle_coordinates == starting_position:
                next_spot_coordinates = get_coordinates_for_spot_ahead(position, direction)
                im_obstacle_coordinates = next_spot_coordinates
                        
            # next_obstacle_hitting_site = obstacle_hitting_side[direction]
            add_obstacle_to_map(im_obstacles_map, im_obstacle_coordinates)                
            add_list_to_map(im_obstacles_history_map, im_obstacle_coordinates)
            update_obstacle_history_map(obstacle_history_map=im_obstacles_history_map, \
                                        obstacle_coordinates = im_obstacle_coordinates, \
                                        guard_coming_from = direction)
            if debug:
                print ('>>>>>>>>>updated im history<<<')

            count_position=True
            if [im_obstacle_coordinates] in all_im_obstacles_position_history:
                count_position=False
            else:
                all_im_obstacles_position_history += [im_obstacle_coordinates]
            

            last_real_position, last_real_direction = position, direction
            direction = turn_right[direction]
            imaginary_mode = True
            if debug:
                print(f'------1----{imaginary_mode=}')
        
        if imaginary_mode:
            im_iterations = 0
            max_im_iterations = 5000
            if debug:
                max_im_iterations = 30
            while (not guard_will_get_out_of_field_next_step(im_obstacles_map, position, direction)) \
                and imaginary_mode and (im_iterations < max_im_iterations):
                im_iterations += 1
                if im_iterations%10000==0:
                    print ('im:',im_iterations)
                
                if debug:
                    print (f'+-+-+-+     {im_iterations=}  +-+-+-+')    
                if there_is_obstacle_ahead(im_obstacles_map, position, direction):
                    if debug:
                        print ('obstacle in imaginary map ahead--------------+++++++++++********')    
                        print (f'im guard located at {position=}, looking at {direction=}')                    
                    next_obstacle_coordinates = get_next_obstacle_ahead_coordinates(im_obstacles_map, \
                                                position, \
                                                direction)
                    if debug:
                        print (f'obstacle located at {next_obstacle_coordinates}')    
                                        
                    next_obstacle_row = next_obstacle_coordinates[0]
                    next_obstacle_col = next_obstacle_coordinates[1]
                    next_obstacle_hitting_side = obstacle_hitting_side[direction]
                    next_obstacle_hitting_side_history = im_obstacles_history_map[next_obstacle_row][next_obstacle_col]
                    if debug:
                        print (f'obstacle history: {next_obstacle_hitting_side_history=}, now: {next_obstacle_hitting_side=}')    

                    if next_obstacle_hitting_side[0] in next_obstacle_hitting_side_history:
                        if count_position:
                            num_of_positions_that_trap_guard+=1
                        if debug:
                            print('Got 1<<<<<<<<<<<<<<<<<YEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE')
                        imaginary_mode = False
                        if debug:
                            print(f'--------2-{imaginary_mode=}')
                    else:
                        update_obstacle_history_map(obstacle_history_map=im_obstacles_history_map, \
                                            obstacle_coordinates=next_obstacle_coordinates, \
                                            guard_coming_from=direction)

                    position = get_new_guard_coordinates_before_next_obstacle_turned_right( \
                                                next_obstacle_coordinates=next_obstacle_coordinates, 
                                                direction=direction)
                    direction = turn_right[direction] 
                    if debug and not imaginary_mode:
                        print('Updated position before next obstacle in imaginary world:')
                        print(f'Guard at {position} looking at {direction}')

                else: 
                    imaginary_mode = False
                    if debug:
                        print(f'---------3{imaginary_mode=}') 
                        

        if not imaginary_mode:
            if debug:
                print ('>>>>>>>>>back to reality, restoring map and map history<<<<<<<')        
            im_obstacles_map = copy.deepcopy(real_obstacles_map)
            im_obstacles_history_map = copy.deepcopy(real_obstacles_history_map)

        position, direction = last_real_position, last_real_direction
        imaginary_mode = False
        if debug:
            print(f'---------4{imaginary_mode=}')
        position = get_coordinates_for_spot_ahead(position, direction)
        if debug:
            print (f'took a step ahead to {position}')
        last_real_position = position
        num_of_real_steps_checked+=1
    
    return num_of_positions_that_trap_guard




In [35]:
test_map2 = [['.','#','.','.','.'],
             ['.','.','#','.','.'],
             ['.','.','.','.','#'],
             ['.','v','.','.','.'],
             ['.','#','.','#','.']
]




position = [3,1]
direction = '^'
check_map(test_map2, debug=True)

----0-----imaginary_mode=False
num_of_real_steps_checked=0
direction='v',position=[3, 1]
real guard at position=[3, 1] looking towards v, towards [4, 1]
obstacle ahead
turned <
>>>>>>>>>no obstacle ahead, adding one<<<<<<<
>>>>>>>>>updated im history<<<
------1----imaginary_mode=True
+-+-+-+     im_iterations=1  +-+-+-+
obstacle in imaginary map ahead--------------+++++++++++********
im guard located at position=[3, 1], looking at direction='^'
obstacle located at [0, 1]
obstacle history: next_obstacle_hitting_side_history=[], now: next_obstacle_hitting_side=['down']
+-+-+-+     im_iterations=2  +-+-+-+
obstacle in imaginary map ahead--------------+++++++++++********
im guard located at position=[1, 1], looking at direction='>'
obstacle located at [1, 2]
obstacle history: next_obstacle_hitting_side_history=[], now: next_obstacle_hitting_side=['left']
+-+-+-+     im_iterations=3  +-+-+-+
obstacle in imaginary map ahead--------------+++++++++++********
im guard located at position=[1, 1]

1

In [40]:
check_map()

num_of_real_steps_checked=0
num_of_real_steps_checked=700
num_of_real_steps_checked=1400
num_of_real_steps_checked=2100
num_of_real_steps_checked=2800
num_of_real_steps_checked=3500
num_of_real_steps_checked=4200
num_of_real_steps_checked=4900


1825

In [7]:
answer1_tried = [4677, 'too_high']
#answer2_calcualted = [4677, 'too_high'] #removed duplicate positions (... maybe there weren't any..)
answer3_tried = [1082, 'too_low'] #added step update in imaginary world
answer3_tried = [2632, 'too_high']
answer4_tried = [1825, '????????']
answer4_tried = [1824, '????????']


In [13]:
a = '^'
b = a
b = '>'
b

'>'

In [37]:
###########error1:#################
test_map1 = [['.','#','#','.','.'],
             ['.','.','.','.','.'],
             ['#','>','.','#','.'],
             ['#','.','.','#','.'],
             ['.','#','#','.','.']
]
check_map(test_map1, debug=True)

----0-----imaginary_mode=False
num_of_real_steps_checked=0
direction='>',position=[2, 1]
real guard at position=[2, 1] looking towards >, towards [2, 2]
>>>>>>>>>no obstacle ahead, adding one<<<<<<<
>>>>>>>>>updated im history<<<
------1----imaginary_mode=True
+-+-+-+     im_iterations=1  +-+-+-+
obstacle in imaginary map ahead--------------+++++++++++********
im guard located at position=[2, 1], looking at direction='v'
obstacle located at [4, 1]
obstacle history: next_obstacle_hitting_side_history=[], now: next_obstacle_hitting_side=['up']
+-+-+-+     im_iterations=2  +-+-+-+
obstacle in imaginary map ahead--------------+++++++++++********
im guard located at position=[3, 1], looking at direction='<'
obstacle located at [3, 0]
obstacle history: next_obstacle_hitting_side_history=[], now: next_obstacle_hitting_side=['right']
+-+-+-+     im_iterations=3  +-+-+-+
obstacle in imaginary map ahead--------------+++++++++++********
im guard located at position=[3, 1], looking at direction='^

4

In [19]:

###########error1:#################
test_map1 = [['.','#','#','.','.'],
             ['.','.','#','#','.'],
             ['.','.','.','.','.'],
             ['#','.','.','.','#'],
             ['.','#','#','.','.']
]

position = [3,1]
direction = '>'
obst_position = get_next_obstacle_ahead_coordinates(test_map1, \
                                                position, \
                                                direction)

new_guard_position=get_new_guard_coordinates_before_next_obstacle_turned_right(obst_position, 
                                                direction)

print ('guard was at:      ', position ,'looking at', direction)
print ('obstacle at ', obst_position)
print ('guard new position:', new_guard_position)


guard was at:       [3, 1] looking at >
obstacle at  [3, 4]
guard new position: [3, 2]


In [47]:
map1 = [['.','#','#','.','.'],
        ['.','.','.','.','.'],
        ['.','.','.','.','.'],
        ['#','.','<','.','.'],
        ['.','#','#','.','.']
]


map2 = [['.','#','#','.','.'],
        ['.','.','.','#','.'],
        ['.','.','.','.','.'],
        ['.','.','.','<','#'],
        ['.','#','#','.','.']
]

map3 = [['.','#','#','.','.'],
        ['.','v','#','.','.'],
        ['#','.','.','.','.'],
        ['#','.','.','.','#'],
        ['.','.','#','.','.']
]

map4 = [['.','#','#','.','.'],
        ['.','.','#','#','.'],
        ['.','.','.','.','.'],
        ['#','.','.','.','#'],
        ['.','#','#','.','.']
]

check_map(map3)

num_of_real_steps_checked=0


2

In [52]:
initial_data = '''....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
'''

map0 = [1,2,3,4,5,6,7,8,9,10]
for i,line in enumerate(initial_data.split()):
    map0[i]=[k for k in line]
map0
check_map(map0)


num_of_real_steps_checked=0


6